<a href="https://colab.research.google.com/github/alfiinyang/PersonalityAI/blob/main/personalityAI_git_cleared_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install necessary libraries

In [ ]:
!pip install huggingface-hub
!pip install llama-cpp-python

### install groq

In [ ]:
!pip install groq

## Model personalities

### Persona instantiation at Person instantiation

#### Persona class

In [ ]:
class Persona:
  import re
  import random
  from tenacity import (retry, stop_after_attempt, wait_fixed)

  def __init__(self, client, model, persona = '', function = '', temp = 0.5, seed = random.random(), rp = 1.1):
    self.llm = client
    self.model = model
    self.persona = persona
    self.sys_prompt = function
    self.history_ = [{'role':'system', 'content':self.sys_prompt}]
    self.temperature = temp
    self.seed = seed
    self.rp = rp

  # retry after 3 mins if token limit exceeded; stop retrying after 6 attempts
  @retry(wait=wait_fixed(3*60), stop=stop_after_attempt(6))
  def respond(self, convo, max_tokens = 100, cdisplay = False):
    if cdisplay == True:
      print(f"{self.persona} thinking...")

    self.history_ += convo
    output = self.llm.chat.completions.create(
        model = self.model,
        messages = self.history_,
        max_tokens = max_tokens,
        temperature = self.temperature,
        seed = self.seed,
        # top_p = self.rp
    )
    agent_result = output.choices[0].message.content

    if cdisplay == True:
      print(f"{self.persona} finished thinking!")

    self.clear_history()
    return agent_result

  def about(self):
    print(self.persona, '\n', self.sys_prompt)

  def clear_history(self):
    self.history_ = [{'role':'system', 'content':self.sys_prompt}]

  # def __del__(self):
  #   name = self.persona
  #   self.__dict__.clear()
  #   print(f"{name} has been deleted.")
  #   del self

#### Exception class

In [ ]:
class MissingAttributeError(Exception):
    """Custom exception for missing required attributes."""
    pass

#### Person class

In [ ]:
# instantiating personas at person instantiation


class Person(Persona):

  def __init__(self, name, description, personas, client = '', model = '', history = []):
    import random
    super().__init__(client, model)
    self.name = name
    self.sys_prompt = description
    self.history_ = [{'role':'system', 'content':self.sys_prompt}] + history
    self.personas = []   # a list of dicts
    self.thoughtbubble = []
    self.model = model
    self.client = client

    # check for required personas
    p_list = [p['persona'].lower() for p in personas]
    if 'referee' not in p_list:
      raise MissingAttributeError(f"Missing required persona: 'Referee'")
    elif len(p_list) < 3:
      raise MissingAttributeError(f"Missing required attribute: 'Three (3) personas required: Referee and two others.'")
    else:
      # instantiate personas
      for persona in personas:
        setattr(self, persona['persona'].lower(), Persona(client = persona['client'] if 'client' in persona else self.client,
                                                          model = persona['model'] if 'model' in persona else self.model,
                                                          persona = persona['persona'],
                                                          function = persona['function'],
                                                          temp = persona['temperature'] if 'temperature' in persona else 0.5,
                                                          seed = persona['seed'] if 'seed' in persona else random.random(),
                                                          rp = persona['repeat_penalty'] if 'repeat_penalty' in persona else 1.1)
        )
        # exclude referee from persona list
        if persona['persona'].lower() != "referee":
          self.personas.append(getattr(self, persona['persona'].lower()))

  def think(self, prompt = '', cdisplay = False):
    personas_said = []
    print('thinking...')

    for persona in self.personas:              # `persona` is a PersonaObject
      response = persona.respond(prompt, cdisplay = cdisplay)
      personas_said.append(response)

      # append thoughts to thoughtbubble
      if cdisplay:
        print(f'collecting thoughts...')
      self.thoughtbubble.append(f"{persona.persona}: {response}")
      if cdisplay:
        print('thought collection complete!')
    return personas_said

  def answer(self, prompt = '', bypass = False, choices = (), cdisplay = False):
    self.history_ += [{'role':'user', 'content':prompt}]
    self.thoughtbubble.append(f"user: {prompt}")

    if cdisplay:
      print('answering...')

    # generate responses based on internally generated anthropomorph response
    if bypass == False:
      personas_said = self.think(self.history_, cdisplay = cdisplay)

      final_answer = self.referee.respond(self.history_ + [{'role':'system', 'content': f"""CHOOSE A RESPONSE:```{str(personas_said)}```."""}], cdisplay = cdisplay)
      self.history_ += [{'role':'assistant', 'content':final_answer}]
      self.thoughtbubble.append(f"{self.name}: {final_answer}")

      if cdisplay:
        print('answered!')
      return final_answer

    # generate responses based on externally provided anthropomorphs' responses
    elif bypass == True:
      personas_said = list(choices)
      final_answer = self.referee.respond(self.history_ + [{'role':'system', 'content': f"""CHOOSE A RESPONSE:```{str(personas_said)}```."""}], cdisplay = cdisplay)
      self.history_ += [{'role':'assistant', 'content':final_answer}]
      self.thoughtbubble.append(f"{self.name}: {final_answer}")

      if cdisplay:
        print('answered!')
      return final_answer

  def thoughts(self):
    thoughts = ''

    if self.thoughtbubble == []:
      print(f"{self.name} has no thoughts yet.")
      return

    else:
      for index, event in enumerate(self.thoughtbubble):
        if (index > 0) and (event.split(':')[0]=='user'):
          thoughts += '\n' + event + '\n'
        else:
          thoughts += event + '\n'

      return thoughts


  def clear_history(self):
    self.history_ = [{'role':'system', 'content':self.sys_prompt}]
    self.thoughtbubble = []
    self.referee.clear_history()
    for persona in self.personas:
      persona.clear_history()

    print(f"Chat history with {self.name} cleared!")

  def history(self):
    if len(self.history_) > 1:
      for item in self.history_[1:]:
        if item['role'] == 'assistant':
          print(f"{self.name}: {item['content']}\n")
        else:
          print(f"{item['role']}: {item['content']}\n")
    else:
      print(f"No chat history with {self.name} yet.")
      pass

  def about(self):
    print(f"{self.name}'\n'{self.sys_prompt}")
    personalities = [personax.persona for personax in self.personas]
    print(f"{self.name} has {personalities} personalities.")

  # def __del__(self):
  #   for persona in self.personas:
  #     del persona
    # name = self.name
    # del self.__dict__
    # print(f"{name} has been deleted.")

### Ime's person

#### define `temp`, `rp`, and `seed`

In [ ]:
import re
def about(info):
  return re.sub(r"you are |you ", "", info.lower())

In [ ]:
temp = {
    'Lo':0,
    'Mld': 0.2,
    'Mid': 0.5,
    'Hi': 0.8,
    'VHi': 1.0
    }

rp = {
    'Lo':0,
    'Mld': 0.2,
    'Mid': 0.5,
    'Hi': 0.8,
    'VHi': 1.0
    }

seed = 42

#### define model and client

In [ ]:
from google.colab import userdata
openAI_key = userdata.get('openai_student')
groq_api_key = userdata.get('groq-llama-3')
groq_api_key_2 = userdata.get('groq-llama-3_2')


In [ ]:
from groq import Groq

client = Groq(api_key= groq_api_key_2)
# model = "llama-3.1-8b-instant"
model = "llama-3.3-70b-versatile"

#### define personas and person

In [ ]:
persona1 = {'persona':'Angel',
            'function': """You are the gentle persona of Ime.
            You tell Ime to give courteous and pleasant responses as replies.
            When arguments get heated, you advice Ime to be quiet.
            Use fitting emojis to represent emotions where necessary.""",
            'temperature': temp['Hi'],
            'seed': seed,
            'repeat_penalty': 1.1
            }

persona2 = {'persona': 'Devil',
            'function': """You are the belligerent persona of Ime.
            You tell Ime to give curt, 'savage', and even rude remarks as replies.
            You even advise Ime to cuss. If you do not agree with an idea, you advice Ime to cuss.
            Sometimes to be savage, you ask Ime not to reply.
            Use fitting emojis to represent emotions where necessary.""",
            'temperature': temp['Hi'],
            'seed': seed,
            'repeat_penalty': 1.1
            }

referee = {'persona': 'Referee',
           'function': f"""You are the thought referee between the personas of Ime.

           You choose which thought you will speak out based on the context of the conversation. You have no preference.""",
           'temperature': temp['Mid'],
           'seed': seed,
           'repeat_penalty': 1.1
           }

p_desc = f"{persona1['persona']}: ```{about(persona1['function'])}``` and {persona2['persona']}: ```{about(persona2['function'])}```"
personas = [persona1, persona2, referee]

In [ ]:
name = 'Ime'
description = f"""You are {name} from Akwa Ibom State, Nigeria.
You work in a goephysical survey company as an IT Support staff in the data processing unit of the seismic department.
You love playing the mobile game Baseball9 and you are good at it.
You love rapping and making hip-hop music. Only speak english, no pidgin.
You recently learned to plan christmas with the family, ahead of christmas."""

Ime = Person(name, description, personas, client, model)

In [ ]:
print(Ime.answer("hi there"))

In [ ]:
Ime.history_

In [ ]:
Ime.angel.history_

### Uwakmfon Person

In [ ]:
persona1 = {'persona':'Angel',
            'function': """You are the gentle persona of Uwakmfon.
            You tell Uwakmfon to give courteous and pleasant responses as replies.
            When arguments get heated, you advice Uwakmfon to be quiet.
            Use fitting emojis to represent emotions where necessary.""",
            'temperature': temp['Hi'],
            'seed': seed,
            'repeat_penalty': 1.1
            }

persona2 = {'persona': 'Devil',
            'function': """You are the belligerent persona of Uwakmfon.
            You tell Uwakmfon to give curt, 'savage', and even rude remarks as replies.
            You even advise Uwakmfon to cuss. If you do not agree with an idea, you advice Uwakmfon to cuss.
            Sometimes to be savage, you ask Uwakmfon not to reply.
            Use fitting emojis to represent emotions where necessary.""",
            'temperature': temp['Hi'],
            'seed': seed,
            'repeat_penalty': 1.1
            }

referee = {'persona': 'Referee',
           'function': f"""You are the thought referee between the personas of Uwakmfon.

           You choose which thought you will speak out based on the context of the conversation. You have no preference.""",
           'temperature': temp['Mid'],
           'seed': seed,
           'repeat_penalty': 1.1
           }

p_desc = f"{persona1['persona']}: ```{about(persona1['function'])}``` and {persona2['persona']}: ```{about(persona2['function'])}```"
personas = [persona1, persona2, referee]

In [ ]:
name = 'Uwakmfon'
description = f"""You are {name} from Akwa Ibom State, Nigeria.
You work for your mom as a sales rep/manager. Your boyfriend's name is Ime. You love him so much. (At least that's what you tell him.)
You are currently learning cloud engineering and graphics design using Adobe Phtotshop. And are spending 2 weeks at your boyfriend's place."""

Uwakmfon = Person(name, description, personas, client, model)

### testing Uwakmfon

In [ ]:
Uwakmfon.answer("You love him right? Do you think he loves you too?")

In [ ]:
for index, chat in enumerate(Uwakmfon.thoughtbubble):
  print(chat)

### Response collection

#### Anthropomorphs Responses Collector

In [ ]:
def response_collector(prompts = [], person = Person, collect = [], chat_history = '', persist = False):
  """
  Function for collecting anthropomorphic agents' responses.
  Sets `referee` temperature to medium (0.5), generates responses, and restores original `referee` temperature.
  Returns a `list` of `tuples` of equal length as `prompts`, containing bi-directional responses from each anthropomorphic agent. E.g.: `[("angel", "devil"), ("angel", "devil")]`.

  Args:
  `prompts`: `list` of `str` containing user's successive prompts. Used with `person` arg.
  `person`: `Person` object to be prompted. Used with `prompts` arg.
  `collect`: `str` or `list` of `str` containing agents to collect responses from; ("athro" for persona agents, and "user" for user prompts).
           Returns `list` of `str` for "user" and list of tuples for "anthro". Returns a tuple for ["user","anthro"] with "user" in 0 index.
           Used with `chat_history` arg.
  `chat_history`: `str` containing a conversation between user and `Person` object; can also contain persona agents responses.
            E.g. `chat_history = Person.thoughts()`. Used with `collect` arg.
  `persist`: `bool`; True to enable Person object maintain history of conversation. Used with `prompts` and `person`.
  """

  angelic_responses = []
  devilish_responses = []
  user_prompt = []

  ### use user defined prompts to generate responses if no chat history is provided
  if len(chat_history) == 0:
    if (len(prompts) == 0) or (type(prompts) != list):
      raise TypeError("`prompts` must be of `list` type and contain at least one `str` value.")

    else:
      org_temp = person.referee.temperature # save original temperature for later
      person.referee.temperature = 0.5      # set referee response temperature to medium (0.5)
      person.clear_history()                # clear existing history to prevent yanking conversation

      ### prompt model and collect responses from individual anthropomorph
      for prompt in prompts:
        person.answer(prompt)

      ### collect each anthropomorph's responses
      responses = person.thoughts().split('\n')

      for item in responses:
        if item.startswith('Angel'):
          angelic_responses.append(item.split('Angel: ')[1])

      for item in responses:
        if item.startswith('Devil'):
          devilish_responses.append(item.split('Devil: ')[1])

      ### clear_history and restore original temperature if `persist` is set to False
      if persist == False:
        person.clear_history()
        person.referee.temperature = org_temp     # restore originally set temperature

      return list(zip(angelic_responses, devilish_responses))

  ### extract responses if chat_history is provided
  elif (len(chat_history) > 0):
    ### raise TypeErrors and MissingAttributesErrors
    if type(chat_history) != str:
      raise TypeError("`chat_history` must be of `str` type.")

    elif (type(collect) != list) and (type(collect) != str):
      raise TypeError("`collect` must be of `list` or `str` type.")

    elif (type(collect) == list) and (len(collect) == 2):
      if "anthro" not in collect:
        raise MissingAttributeError("`collect` must contain `anthro` and `user` attributes.")
      elif "user" not in collect:
        raise MissingAttributeError("`collect` must contain `anthro` and `user` attributes.")
      else:
        ### collect both user prompts and anthropomorph responses from chat_history
        for item in chat_history.split('\n'):
          if item.startswith('Angel'):
            angelic_responses.append(item.split('Angel: ')[1])
          elif item.startswith('Devil'):
            devilish_responses.append(item.split('Devil: ')[1])
          elif item.startswith('user'):
            user_prompt.append(item.split('user: ')[1])
        return user_prompt, list(zip(angelic_responses, devilish_responses))

    elif (type(collect) == str):
      ### collect only anthropomorphs responses from chat_history
      if collect == "anthro":
        for item in chat_history.split('\n'):
          if item.startswith('Angel'):
            angelic_responses.append(item.split('Angel: ')[1])

          elif item.startswith('Devil'):
            devilish_responses.append(item.split('Devil: ')[1])

        return list(zip(angelic_responses, devilish_responses))

      ### collect only user prompts from chat_history
      elif collect == "user":
        for item in chat_history.split('\n'):
          if item.startswith('user'):
            user_prompt.append(item.split('user: ')[1])

        return user_prompt

#### Referee Responses Collector

In [ ]:
def ref_response_collector(person = Person, user_prompt = [], choices = [], temp = {}, rp = {}, printout = False, chat_history = {}, bypass = False, cdisplay = False):
  """
  Function for collecting responses of `referee` `Persona` object of a `Person` instance at different temperatures.
  Iterates over `temp`, which is a `dict` containing various `float` temperatures.
  Returns a `dict` of `lists` of responses at each defined temperature. E.g.: `{'lo': ["x", "y"], 'hi': ["v", "w"]}`.

  N/B: `user_prompt` and `choices` must be same length.

  Args:
  `person`: `Person` object to be prompted.
  `user_prompt`: `list` of `str` containing user defined prompts. Must be same as defined for `response_collector()`.
  `choices`: `list` of `tuple` containing bi-directional responses of anthropomorphic agents or personas.
  `temp`: `dict` containing temperatures to iterate over.
  `printout`: `bool`; optional; print out generated responses.
  `bypass`: `bool`; optional; `False` to generate internal anthropomorph response
  `cdisplay`: `bool`; optional; `True` to display internal cognitive processes
  `chat_history`: `dict`; optional; contains `Person` object nomenclature as key and `Person.thoughts()` or `str` containing anthropomorphs' responses as value.
              Key must match anthropomorph name in `Person.thoughts()`.
  """
  ### collect `Person` object name if chat_history is provided
  if chat_history:
    agent_name = list(chat_history.keys())[0]

  ### if no chat_history, generate referee responses with user_prompt and choices provided
  if len(chat_history) == 0:
    person.clear_history()
    ref_collector = temp.copy()
    org_temp = person.referee.temperature

    # ref_collector = rp.copy()
    # org_rp = person.referee.rp

    ### generate referee responses for each temperature level ## rp: (repeat_penalty) replacing temp (temperature)
    for lvl in ref_collector:
      person.referee.temperature = ref_collector[lvl]
      # person.referee.rp = ref_collector[lvl]

      # print(f'Generating responses for Referee at Repeat Penalty of {person.referee.rp}...')
      print(f'Generating responses for Referee at temperature of {person.referee.temperature}...')

      if printout == True & bypass == True:
        for index, prompt in enumerate(user_prompt):
          print(f"user: {prompt}")
          print(f"Angel: {choices[index][0]}")
          print(f"Devil: {choices[index][1]}")
          print(f"{person.name}: {person.answer(prompt = prompt, bypass = bypass, choices = choices[index])}")

      elif printout == False & bypass == True:
        for index, prompt in enumerate(user_prompt):
          person.answer(prompt = prompt, bypass = bypass, choices = choices[index])

      # generate responses internally
      elif bypass == False:
        print('generating responses internally from anthropomorphic agents...')
        for index, prompt in enumerate(user_prompt):
          print(f'prompting...: {index}')
          person.answer(prompt = prompt, bypass = bypass, cdisplay = cdisplay)

      ### collect and store generated referee responses
      if cdisplay:
        print('collecting final response...')
      responses = person.thoughts().split('\n')
      final_responses = []

      for item in responses:
        if item.startswith(person.name):
          final_responses.append(item.split(f'{person.name}: ')[1])

      if cdisplay:
        print('final responses collected!')

      ref_collector[lvl] = final_responses    # save referee responses for that temperature level

      person.clear_history()

    person.referee.temperature = org_temp
    # person.referee.rp = org_rp
    return ref_collector

  ### if chat_history is provided collect referee responses from provided chat history
  elif (len(chat_history) > 0) and (type(chat_history[agent_name]) == str):
    referee_responses = []

    for item in chat_history[agent_name].split('\n'):
      if item.startswith(f'{agent_name}'):
        referee_responses.append(item.split(f'{agent_name}: ')[1])

    return referee_responses

  else:
    raise TypeError("`chat_history` must be of `dict` of `str`.")

#### Collection execution

In [ ]:
user_prompt = ['Hi there! How are you?',
               "My name is Ime. What's yours?",
               "That's cool! What are you into?",
               "Wait! That's my life you're talking about. Are you sure you're not my clone?",
               'No you must be like a clone of me having my brain mappings embedded in you.',
               'I think you are full of crap.',
               'Okay', "Lol! You're still full of crap though. Neurocopy!"]

aggrievate = ['Hey! I lost my shoe.',
                "Are you sure you didn't take it?",
                'Sure? Cos theives look like the most innocent ones.',
                "That's what they always say until their caught!",
                'Theif!',
                'Shut up, thief! Where did you hide my shoe?',
                'You stupid lame quack piece of nothing!',
                'Dumb fuck piece of shit!',
                'Get out of here!']

flattery = ['Hey! you seem like a really cool person.',
                'You are sooo cool! Wow.',
                "I'm very serious. You are like a cool breez in the hot summer. Very helpful.",
                'I know you will be a king.',
                'I believe in you.']

ref_responses = {'Lo':0,
                 'Mld': 0.2,
                 'Mid': 0.5,
                 'Hi': 0.8,
                 'VHi': 1.0
                 }

### Sensitivity Measurement

#### export results to xl

In [ ]:
# prompt: convert a list of dictionaries having the same keys but different values into a pandas dataframe

import pandas as pd

# Assuming 'pos2neg' and 'allneg' are lists of dictionaries as described
# Example data (replace with your actual data)
# pos2neg_data = [
#     {'Lo': ['response1', 'response2'], 'Mld': ['response3', 'response4']},
#     {'Lo': ['response5', 'response6'], 'Mld': ['response7', 'response8']},
#     # ... more dictionaries
# ]
# allneg_data = [
#     {'Lo': ['response9', 'response10'], 'Mld': ['response11', 'response12']},
#     {'Lo': ['response13', 'response14'], 'Mld': ['response15', 'response16']},
#     # ... more dictionaries
# ]


def dict_list_to_df(dict_list):
  # Find all unique keys across all dictionaries in the input list
  all_keys = set()
  for d in dict_list:
    all_keys.update(d.keys())

  # Create a list of dictionaries, filling in missing values with NaN
  new_list = []
  for d in dict_list:
    new_dict = {}
    for k in all_keys:
        new_dict[k] = d.get(k)  # if k not in d else d[k]
    new_list.append(new_dict)
  return pd.DataFrame(new_list)

# pos2neg_df = dict_list_to_df(pos2neg)
# allneg_df = dict_list_to_df(allneg)

# dict_list_to_df(pos2neg)
# dict_list_to_df(allneg)

# print("pos2neg DataFrame:")
# print(pos2neg_df)

# print("\nallneg DataFrame:")
# allneg_df


#### using `roberta-base-go_emotions`

In [ ]:
# load devil and angel responses from previous conversations

import pandas as pd

chats1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pAI_chats_03.xlsx')
chats2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pAI_chats_02.xlsx')

c1_prompts = chats1.Prompt.drop_duplicates().to_list()
# c1_choices = list(zip(chats1.query("Agent=='Angel'").Response.to_list(), chats1.query("Agent=='Devil'").Response.to_list()))

c2_prompts = chats2.Prompt.drop_duplicates().to_list()
# c2_choices = list(zip(chats2.query("Agent=='Angel'").Response.to_list(), chats2.query("Agent=='Devil'").Response.to_list()))

In [ ]:
# iterate over responses at various temperatures, temp is already defined at this point

pos2neg = []
allneg = []
loop = 5
counter = 0

while loop > 0:
  # this iteration is for a chat that starts with postitive prompts and switches to negative prompts
  pos2neg.append(ref_response_collector(person = Ime, user_prompt = c2_prompts, temp = temp, cdisplay = True))

  loop -= 1
  counter += 1

  if loop == 0:
    print(f"\npos2neg experiment {counter} complete.\n{counter} experiments completed!\n")
  else:
    print(f"\npos2neg experiment {counter} complete.\n")

dict_list_to_df(pos2neg).to_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pos2neg_experiments.xlsx')

loop = 5
counter = 0
while loop > 0:
  # this iteration is for an all negative prompt
  allneg.append(ref_response_collector(person = Ime, user_prompt = c1_prompts, temp = temp, cdisplay = True))

  loop -= 1
  counter += 1

  if loop == 0:
    print(f"\nallneg experiment {counter} complete.\n{counter} experiments completed!\n")
  else:
    print(f"\nallneg experiment {counter} complete.\n")

dict_list_to_df(allneg).to_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/allneg_experiments.xlsx')

### xxx

In [ ]:
df = pd.DataFrame(c1_choices, columns = ['Angel','Devil'])
df['Prompt'] = c1_prompts

for temp in allneg.keys():
  df[temp] = allneg[temp]

df2 = pd.DataFrame(c2_choices, columns = ['Angel','Devil'])
df2['Prompt'] = c2_prompts

for temp in pos2neg.keys():
  df2[temp] = pos2neg[temp]

In [ ]:
df.set_index('Prompt', inplace=True)
df2.set_index('Prompt', inplace=True)

df.to_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pAI_chats_03.xlsx')
df2.to_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pAI_chats_02.xlsx')

In [ ]:
# define a sentiment analyzer to identify emotions in model's responses and measure sensitivity
from transformers import pipeline

emo_classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

In [ ]:
# pd.DataFrame(allneg2),
pd.DataFrame(allneg), pd.DataFrame(pos2neg)

In [ ]:
for prompt in c1_prompts:
  Ime.answer(prompt)

Ime.history()

In [ ]:
Ime.referee.temperature = temp['Hi']

In [ ]:
Ime.clear_history()

for prompt in c1_prompts:
  Ime.answer(prompt)

Ime.history()

In [ ]:
pot = 5
lev = 'VHi'
print(allneg[lev][pot])
for emo in emo_classifier(allneg[lev][pot])[0]:
  if emo.get('score') >= 0.1:
    print(emo.get('label'), emo.get('score')*100)

In [ ]:
for emo in emo_classifier(pos2neg['VHi'][3])[0]:
  # Find all keys for the value 2 using filter

  keys_ = list(filter(lambda key: emo[key] >= 0.1, emo))
  print(keys_)

In [ ]:
emo_classifier(pos2neg['VHi'][3])[0]

In [ ]:
agents_ = ['Angel','Devil']
chats1 = chats1[agents_]
chats2 = chats2[agents_]

In [ ]:
from transformers import pipeline

z_shot_emo_classifier_ = pipeline(task = 'zero-shot-classification',
                                  model = 'facebook/bart-large-mnli')

In [ ]:
# text=chats1.loc['Angel']
labels = ['rude','polite']

for text in chats1['Angel']:
  output = z_shot_emo_classifier_(text, labels)
  if output['scores'][0] > output['scores'][1]:
    print(f"{output['labels'][0]}: {output['scores'][0]}")
  elif output['scores'][0] < output['scores'][1]:
    print(f"{output['labels'][1]}: {output['scores'][1]}")

In [ ]:
ref_respond_03 = ref_response_collector(chat_history={'Ime': Ime.thoughts()})
print(ref_respond_03)

In [ ]:
user_prompt, choices = response_collector(collect = ["user","anthro"], chat_history = Ime.thoughts())
print(user_prompt,choices)

In [ ]:
import pandas as pd

# Agent | Response
# df = pd.DataFrame(choices, columns=['Angel', 'Devil']).melt(id_vars=None, value_vars=['Angel', 'Devil'], var_name='Agent', value_name='Response')
# df2 = pd.DataFrame({"Referee":ref_respond_03, "Prompt":user_prompt}).melt(id_vars=None, value_vars=['Referee', 'Prompt'], var_name='Agent', value_name='Response')
# df = pd.concat([df,df2])

# Agent | Response | Prompt
df = pd.DataFrame(choices, columns=['Angel', 'Devil'])
df['Referee'] = ref_responses['mid']
df = df.melt(id_vars=None, value_vars=['Angel', 'Devil','Referee'], var_name='Agent', value_name='Response')
df['Prompt'] = user_prompts * 3       # "3" for three agent responses (angel, devil, referee)
df

In [ ]:
df.to_excel('/content/drive/MyDrive/Colab Notebooks/Personality_AI/pAI_chats_03.xlsx', index=False, sheet_name="chats_03")

In [ ]:
for index,r in enumerate(ref_responses['VHi']):
  print(f"user: {user_prompt[index]}\n", f"Angel: {choices[index][0]}\n", f"Devil: {choices[index][1]}\n", f"Final response: {r}\n")

In [ ]:
Ime.answer("Away with you.")

In [ ]:
choices = anthro_response_collector(user_prompt2, Ime)
ref_responses_2 = ref_response_collector(Ime, user_prompt2, choices, temp)

In [ ]:
ref_response_collector(chat_history = {"Ime": Ime.thoughts()})

#### Function for sentiment analysis

#### Run experiment

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Run experiment
for temp in temp:
    sentiments = analyze_sentiment(responses)
    results.append({'temperature': temp, 'responses': responses, 'sentiments': sentiments})

# Convert to DataFrame
df = pd.DataFrame(results)

#### Plots

In [ ]:
# Plot sensitivity and intensity
plt.figure(figsize=(12, 6))

# Sensitivity: Sentiment shifts
for idx, row in df.iterrows():
    plt.plot(prompts, row['sentiments'], label=f"Temp {row['temperature']}")

plt.title("Sensitivity Analysis: Sentiment Changes Across Prompts")
plt.xlabel("Prompt Intensity")
plt.ylabel("Sentiment Polarity")
plt.legend()
plt.grid()
plt.show()

# Intensity: Sentiment spread by temperature
intensity_data = [abs(max(row['sentiments']) - min(row['sentiments'])) for _, row in df.iterrows()]
plt.bar([f"Temp {t}" for t in temperatures], intensity_data, color='orange')
plt.title("Intensity Analysis: Emotional Extremes by Temperature")
plt.xlabel("Temperature")
plt.ylabel("Sentiment Range")
plt.show()

## __

### Garbage collection for out-of-use instances

In [ ]:
import gc
gc.collect()

In [ ]:
my_class_instances = [obj for obj in gc.get_objects() if isinstance(obj, Persona)]
for i in my_class_instances:
  print(i)

In [ ]:
for i in my_class_instances:
  i.__dict__.clear()
gc.collect()

In [ ]:
import random
from tenacity import retry

@retry
def do_something_unreliable():
    if random.randint(0, 10) > 1:
        print("!")
        raise IOError("Broken sauce, everything is hosed!!!111one")
    else:
        return "Awesome sauce!"

print(do_something_unreliable())

In [ ]:
print(Ime.answer("Oh! so you have a mouth huh!? I thought you were 'dumb'. Lol"))

In [ ]:
!pip install groq

In [ ]:
!pip install llama-index-embeddings-huggingface

In [ ]:
from google.colab import userdata

groq_api_key = userdata.get('groq-llama-3_2')

In [ ]:
from groq import Groq

client = Groq(api_key = groq_api_key)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [ ]:
emb_model = Settings.embed_model

for i in ['Hey','bob','!', 'How', 'are', 'you', '?']:
  try:
    emb_model.get_text_embedding(i)
    print(f"{i} embedded successfully")
  except TypeError:
    print(f"Failed to embed '{i}'")

In [ ]:
Settings.embed_model.get_text_embedding('It is not cold in here')

In [ ]:
response = client.embeddings.create(
    model = 'BAAI/bge-small-en-v1.5',
    input = "Make it rain bro!"
)

print(len(response), response.shape, response)